# CS4263 Final Project:
## Using Google Search Volume and News Sentiment to Predict Natural Gas Prices with LSTMs
by Quinn Murphey, Adrian Ramos, and Gabriel Soliz

In [63]:
import pandas as pd
import math
from datetime import date
from dateutil.relativedelta import relativedelta

# Data Fetch Stage

### Google Trends Scraper

In [111]:
import pytrends
from pytrends.request import TrendReq
import time
pytrend = TrendReq()

def get_daily_trends_as_df(keywords, categories, timeframe, countries, search_type):
    # Get pytrend suggestions and store them in exact_keywords
    keywords_codes = [pytrend.suggestions(keyword=i)[0] for i in keywords] 
    df_CODES= pd.DataFrame(keywords_codes)
    exact_keywords = df_CODES['mid'].to_list()

    # Store keywords alongside their respective categories
    individual_exact_keyword = list(zip(*[iter(exact_keywords)]*1, categories))
    individual_exact_keyword = [list(x) for x in individual_exact_keyword]

    # Split timeframe into 6 month chunks
    timeframe_start, timeframe_end = timeframe.split(" ")
    start_date = date.fromisoformat(timeframe_start)
    end_date = date.fromisoformat(timeframe_end)
    dates = []
    while start_date < end_date:
        dates.append(start_date)
        start_date = start_date + relativedelta(months=6)
    dates.append(end_date + relativedelta(days=1))

    # Compute number of fetches required
    max_fetches = len(countries) * len(keywords) * (len(dates) - 1)
    curr_fetches = 0

    trend_dict = {}
    for country in countries:
        for keyword, category in individual_exact_keyword:
            trend_dict[keyword] = pd.DataFrame()
            for i in range(len(dates)-1):
                pytrend.build_payload(kw_list=[keyword], 
                                    timeframe = dates[i].isoformat() + " " + (dates[i + 1] - relativedelta(days=1)).isoformat(), 
                                    geo = country, 
                                    cat=category,
                                    gprop=search_type)
                curr_fetches += 1
                print("[" + "=" * math.floor(20 * curr_fetches / max_fetches - 1) +  ">" * min(math.floor(20 * curr_fetches / max_fetches), 1) + " " * (20 - math.floor(20 * curr_fetches / max_fetches)) + "] ",end='')
                print(f"{math.floor(10000 * curr_fetches / max_fetches)/100}% Complete!", end='\r')
                time.sleep(15) # sleep to prevent google shutting us down
                trend_dict[keyword] = pd.concat([trend_dict[keyword], pytrend.interest_over_time()], axis=0)
            i+=1
    df_trends = pd.concat(trend_dict, axis=1)

    df_trends.columns = df_trends.columns.droplevel(0) #drop outside header
    df_trends = df_trends.drop('isPartial', axis = 1) #drop "isPartial"
    df_trends.reset_index(level=0,inplace=True) #reset_index
    df_trends.columns = ['date'] + keywords

    return df_trends

In [116]:
KEYWORDS            = ["Natural Gas","Oil","Coal","Nuclear Power","Wind Power","Hydroelectric","Solar Power","Gold","Silver","Platinum","Copper","Biofuel","Recession","CPI"]
KEYWORDS_CATEGORIES = [904,          904,  904,   0,               0,          0,               0,           904,   904,     904,       904,     0,        0,          0]
TIMEFRAME='2013-01-01 2019-06-30' # Jan 2013 - June 2019
COUNTRIES=["US"] # ISO country code
SEARCH_TYPE='' #default is 'web searches',others include 'images','news','youtube','froogle' (google shopping)

try:
    google_trends_df = pd.read_csv("data/google_trends_dataset.csv")
    google_trends_df = google_trends_df.drop(google_trends_df.columns[0], axis=1)
except:
    google_trends_df = get_daily_trends_as_df(KEYWORDS, KEYWORDS_CATEGORIES, TIMEFRAME, COUNTRIES, SEARCH_TYPE)
    google_trends_df.to_csv("data/google_trends_dataset.csv")

google_trends_df

,date,Natural Gas,Oil,Coal,Nuclear Power,Wind Power,Hydroelectric,Solar Power,Gold,Silver,Platinum,Copper,Biofuel,Recession,CPI
0,2013-01-01,8,37,0,18,43,15,40,18,35,33,27,34,53,16
1,2013-01-02,62,95,0,20,60,28,52,23,50,31,72,47,49,55
2,2013-01-03,54,81,0,23,52,18,56,23,45,15,42,55,41,45
3,2013-01-04,68,78,27,19,54,23,51,23,43,100,58,37,51,45
4,2013-01-05,7,68,0,15,33,11,46,23,34,36,54,15,25,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2367,2019-06-26,25,44,0,41,20,6,67,87,75,43,79,31,35,59
2368,2019-06-27,28,56,0,35,30,23,66,77,74,68,68,22,44,60
2369,2019-06-28,30,55,0,38,20,10,66,79,67,79,73,31,38,50
2370,2019-06-29,8,28,59,37,18,15,69,59,58,44,67,14,27,25


# Data Preprocessing Stage

# Model Training Stage

# Model Testing Stage